In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.insert(0, '..')

from pairing import Reader, Extractor, Classifier
import definition
import numpy as np
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import f1_score, confusion_matrix

## Data Loading and Preprocessing

In [2]:
train_ratio = 0.5

raw_data = Reader.read_file(definition.DATA_PAIRED_SAMPLE)
index = np.arange(len(raw_data))
np.random.shuffle(index)
index_train = index[:int(train_ratio * len(raw_data))]
index_test = index[int(train_ratio * len(raw_data)):]
raw_data_train = np.array(raw_data)[index_train].tolist()
raw_data_test = np.array(raw_data)[index_test].tolist()

data_train = Extractor.extract_data(raw_data_train)
data_test = Extractor.extract_data(raw_data_test)

100%|███████████████████████████████████████████████████████████████████████████| 1349/1349 [00:00<00:00, 46547.40it/s]


In [3]:
data_train.head()

,dist_endpoint,dist_start,len_aspect_char,len_aspect_word,len_sentiment_char,len_sentiment_word,position_aspect,position_sentiment,target
0,2,3,10,2,5,1,0,3,1
1,5,6,10,2,4,1,0,6,1
2,7,8,10,2,14,2,0,8,1
3,12,13,10,2,6,1,0,13,0
4,8,8,8,2,5,1,11,3,0


In [4]:
data_train.describe()

,dist_endpoint,dist_start,len_aspect_char,len_aspect_word,len_sentiment_char,len_sentiment_word,position_aspect,position_sentiment,target
count,9904.000000,9904.000000,9904.000000,9904.000000,9904.000000,9904.000000,9904.000000,9904.000000,9904.000000
mean,8.817649,9.174374,7.872779,1.326939,8.211632,1.497375,12.112076,13.257169,0.303615
std,10.058170,10.074126,3.653968,0.580522,4.343831,0.707566,13.794936,13.862935,0.459841
min,1.000000,1.000000,2.000000,1.000000,2.000000,1.000000,0.000000,0.000000,0.000000
25%,2.000000,2.000000,5.000000,1.000000,5.000000,1.000000,3.000000,3.000000,0.000000
50%,5.000000,6.000000,7.000000,1.000000,6.000000,1.000000,8.000000,8.000000,0.000000
75%,12.000000,12.000000,10.000000,2.000000,11.000000,2.000000,17.000000,18.000000,1.000000
max,81.000000,81.000000,34.000000,6.000000,33.000000,6.000000,81.000000,82.000000,1.000000


In [5]:
X_train = data_train.drop('target', axis=1)
X_test = data_test.drop('target', axis=1)
y_train = data_train['target']
y_test = data_test['target']

## Training

In [6]:
model = Classifier()

In [7]:
model.fit(X_train, y_train)
model.model

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        importance_type='split', learning_rate=0.1, max_depth=-1,
        min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
        n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
        random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [8]:
pred = model.predict(X_test)

In [9]:
print("f1_0 : {}".format(f1_score(y_test, pred, pos_label=0)))
print("f1_1 : {}".format(f1_score(y_test, pred, pos_label=1)))
print("f1_a : {}".format(f1_score(y_test, pred, average='macro')))

f1_0 : 0.9412052515697626
f1_1 : 0.8713648528099912
f1_a : 0.9062850521898769


In [10]:
Classifier.generate_confusion_matrix_table(y_test, pred)

,predicted_0,predicted_1
true_0,5771,251
true_1,470,2442
